## Init

In [1]:
repo = "https://github.com/ventus550/HandwritingRecognition"
! ! (git status 2> /dev/null) && git clone {repo}
%cd ShapeCorrection

On branch master
Your branch is ahead of 'origin/master' by 1 commit.
  (use "git push" to publish your local commits)

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
  (commit or discard the untracked or modified content in submodules)
	modified:   ../TkCanvas (modified content, untracked content)

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	ETL_app.ipynb

no changes added to commit (use "git add" and/or "git commit -a")
[Errno 2] No such file or directory: 'ShapeCorrection'
/home/ventus/Documents/HandwritingRecognition/notebooks


In [2]:
import sys
sys.path.append("..")

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tqdm import tqdm
from pathlib import Path
from contextlib import suppress

2023-05-17 02:32:04.128167: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-17 02:32:04.265044: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-17 02:32:04.267767: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-17 02:32:04.267777: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

## Configuration

In [4]:
BASEDIR = Path().absolute().parent
DESTDIR = BASEDIR / "data"
SRCDATA = DESTDIR / "app"
SUBDIRS = [
    "english/continous",
    "english/separated",
    "unknown/continous"
]

## Load raw data

In [5]:
def read_directory(dir: Path, data_format="png"):
    images, labels = [], []
    for path in dir.iterdir():
        image = tf.image.decode_png(tf.io.read_file(str(path)), 1)
        label = path.name.removesuffix(f".{data_format}")
        images.append(image)
        labels.append(label)

    images, labels = zip(*sorted(zip(images, labels), key=lambda x: x[1]))
    return np.array(images, "float32") / 255.0, np.array(labels)

In [6]:
X, Y = read_directory(SRCDATA / "english" / "separated")

2023-05-17 02:32:06.167761: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-05-17 02:32:06.167786: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-05-17 02:32:06.167807: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (fedora): /proc/driver/nvidia/version does not exist
2023-05-17 02:32:06.168108: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
set(X.ravel())

{0.0,
 0.003921569,
 0.007843138,
 0.011764706,
 0.015686275,
 0.019607844,
 0.023529412,
 0.02745098,
 0.03137255,
 0.03529412,
 0.039215688,
 0.043137256,
 0.047058824,
 0.050980393,
 0.05490196,
 0.05882353,
 0.0627451,
 0.06666667,
 0.07058824,
 0.07450981,
 0.078431375,
 0.08235294,
 0.08627451,
 0.09019608,
 0.09411765,
 0.09803922,
 0.101960786,
 0.105882354,
 0.10980392,
 0.11372549,
 0.11764706,
 0.12156863,
 0.1254902,
 0.12941177,
 0.13333334,
 0.13725491,
 0.14117648,
 0.14509805,
 0.14901961,
 0.15294118,
 0.15686275,
 0.16078432,
 0.16470589,
 0.16862746,
 0.17254902,
 0.1764706,
 0.18039216,
 0.18431373,
 0.1882353,
 0.19215687,
 0.19607843,
 0.2,
 0.20392157,
 0.20784314,
 0.21176471,
 0.21568628,
 0.21960784,
 0.22352941,
 0.22745098,
 0.23137255,
 0.23529412,
 0.23921569,
 0.24313726,
 0.24705882,
 0.2509804,
 0.25490198,
 0.25882354,
 0.2627451,
 0.26666668,
 0.27058825,
 0.27450982,
 0.2784314,
 0.28235295,
 0.28627452,
 0.2901961,
 0.29411766,
 0.29803923,
 0.30196

In [8]:
X.shape

(50, 32, 128, 1)

In [9]:
Y

array(['accident', 'advocate', 'ancestor', 'ball', 'cat', 'ceremony',
       'club', 'conversation', 'convince', 'cord', 'cousin', 'debate',
       'deserve', 'domination', 'driver', 'evoke', 'extension', 'false',
       'fare', 'favourite', 'fear', 'food', 'force', 'glare', 'lease',
       'lesson', 'location', 'lock', 'mainstream', 'matrix', 'meadow',
       'money', 'murder', 'nonremittal', 'offer', 'office', 'overall',
       'overcharge', 'palm', 'peanut', 'premium', 'retain', 'rhetoric',
       'sailor', 'satisfied', 'security', 'see', 'sticky', 'understand',
       'volcano'], dtype='<U12')

## Save data

In [10]:
def save_dataset(source_dir: Path):
	X, Y = read_directory(source_dir)
	dataset_src = str(source_dir.relative_to(DESTDIR))
	dataset_name = "_".join(dataset_src.split("/"))
	np.savez(DESTDIR / dataset_name, images=X, labels=Y)

In [11]:
%%time
for sub in SUBDIRS:
	try:
		save_dataset(SRCDATA / sub)
	except e:
		print(f"Error encoutered in {sub}")
		print(e)

CPU times: user 29.7 ms, sys: 4.5 ms, total: 34.2 ms
Wall time: 44.7 ms
